# TP1
## Grupo 15

Carlos Eduardo Da Silva Machado A96936

Gonçalo Manuel Maia de Sousa A97485

## Problema 2

### Descrição do Problema.

O problema envolve "hard lattices" (reticulados inteiros), um reticulado inteiro pode ser definido pela seguinte matriz $\;\mathsf{L} \in \mathbb{Z}^{m\times n}\;$ (com $\;m > n\;$) de inteiros e por um inteiro primo $\;q\geq 3\;$.
O problema do vetor curto  (SVP) consiste  no cálculo de um vetor de inteiros $\;e\in \{-1,0,1\}^m$ não nulo que verifique a relação $\forall\,i < n\,\centerdot\,$$         $$\sum_{j< m}\,e_j\,\times\,\mathsf{L}_{j,i}\;\equiv\;0\mod q$.

1. Pretende-se resolver o SVP, sendo que:
    - Para m,n e q, $\,n > 30\,$, $\,|m| > 1 + |n|\;$ e $\,|q| > |m|\,$
    - Os elementos $\;\mathsf{L}_{j,i}\;$ são gerados aleatória e uniformemente no intervalo inteiro $\,\{-d \cdots d\}$ sendo  $\;d\equiv (q-1)/2\;$. 
2. Pretende-se determinar:
    - Se existe um vetor $\,e\,$ não nulo (pelo menos um dos $\,e_j\,$é diferente de zero)
    - Se existir $\,e\,$ pretende-se calcular o vetor que minimiza o número de componentes não nulas.

### Abordagem do Problema.

Temos como input do problema as variáveis $n$, $m$, $q$ e a matriz $L_{n,m}$.

Será necessário criar um vetor $e$ que varia de $-1$ a $1$.

Além disso será necessário criar dois vetores adicionais, $Y$ e $k$ de tamanhos $m$ e $n$ respetivamente.

Y é tal que:
$$\forall_{j< m} \quad Y_j = 0 \quad sse \quad e_j = 0$$.
K serve apenas para garantir a divisibilidade por $q$.

Adicionalmente, establelecemos as seguintes restrições:
- O vetor $e$ não pode ser nulo;
- $Y_j$ deve ser zero se e só se $e_j$ é zero em alternativa deve ser 1, para $j<m$;
- $\sum_{j < m} e_j\times L_{j,i}$ é divisível por $q$;


## Código Python

Importação das bibliotecas que iremos utilizar.

In [3]:
from pyscipopt import Model
import random

Funções auxiliares.

In [ ]:
def printmatrix(x,m,n):
    for i in range(m):
        for j in range(n):
            print(x[i,j], end = ' ')  
        print()
        
def check_sol(L,e,q,n,m):
    check = True
    for i in range(n):
        check = check and (sum([e[j]*L[j,i] for j in range(m)]) % q == 0)
    return check

Exemplo 1:

In [7]:
n,m,q,d = 1,8,17,8

Exemplo 2:

In [4]:
n,m,q,d = 3,8,17,8

Exemplo 3:

In [15]:
n,m,q,d = 4,17,37,18

Exemplo 4:

In [26]:
n,m,q,d = 5,20,41,20

Criação do $\textit{model}$:

In [ ]:
model = Model('model')

Criação da matriz $L$ de acordo com o $d$:

In [27]:
L = {}
for j in range(m):
    for i in range(n):
        L[j,i] = random.randrange(-d,d+1)

Declaração do vetor $e$ como um vetor de variáveis inteiras entre $-1$ e $1$:

In [28]:
e = {}
for j in range(m):
    e[j] = model.addVar(f'e[{j}]',vtype = 'I', lb = -1, ub = 1)

Declaração de um vetor de variáveis binárias $Y$ com o mesmo tamanho de $e$ tal que:
    $$\forall_{j< m} \quad Y_j = 0 \quad sse \quad e_j = 0$$

In [29]:
Y = {}
for j in range(m):
    Y[j] = model.addVar(f'Y[{j}]',vtype = 'B')

Declaração de um vetor de variáveis inteiras $k$ de tamanho $n$:

In [30]:
k = {}
for i in range(n):
    k[i] = model.addVar(f'k[{i}]', vtype = 'I')

### Otimizações.

É necessário minimizar o número de componentes diferentes de zero em $e$ isto é feito através do vetor $Y$:

In [31]:
model.setObjective(sum([Y[j] for j in range(m)]),sense = 'minimize')

### Restrições.

O vetor $e$ não pode ser nulo.
Como $Y_j$ representa a existência, ou não, de um elemento não nulo de $e$ em $j < m\;$ esta restrição pode ser feita do seguinte modo:
$$
\sum_{j< m} \;Y_j \ge 1
$$
    
    

In [32]:
model.addCons(sum([Y[j] for j in range(m)]) >= 1)

c1

$Y_j$ deve ser $0$ quando $e_j = 0$ e $1$ quando $e_j = 1$ ou $e_j = -1$.
Logo esta restrição é tal que:
$$
\forall_{j<m} \quad e_j\,^2 = y_j
$$

In [33]:
for j in range(m):
    model.addCons(e[j]*e[j] == Y[j])

Deve existir um valor inteiro, a que chamaremos $k_i$ tal que $e_j\times L_{j,i} = k_i\times q$ para $i<n,j<m$.
Podemos exprimir esta condição da seguinte forma:
$$
\forall_{i < n} \quad \sum_{j < m} e_j\times L_{j,i} = k_i\times q
$$

In [34]:
for i in range(n):
    model.addCons(sum([e[j]*L[j,i] for j in range(m)]) == k[i]*q)

### Resolução dos exemplos.

Comando para esconder informação adicional.

In [ ]:
model.hideOutput()

In [ ]:
Função para resolver e 

In [35]:
def resolve():
    model.optimize()
    if model.getStatus() == 'optimal':
        sol = model.getBestSol()
        x = {}
        for i in range(m):
            x[i] = sol[e[i]]
            print('{}'.format(sol[e[i]]))
        print(check_sol(L,x,q,n,m))
    else:
        print('infeasible')

Exemplo 1:

In [25]:
resolve()

optimal
0.0
0.0
0.0
0.0
0.0
0.0
1.0
-1.0
True


Exemplo 2:

In [14]:
resolve()

0.0
0.0
-1.0
0.0
0.0
0.0
-1.0
1.0
True


Exemplo 3:

In [25]:
resolve()

0.0
1.0
1.0
0.0
0.0
0.0
-1.0
0.0
1.0
-1.0
0.0
-1.0
0.0
1.0
0.0
0.0
0.0
True


Exemplo 4:

In [36]:
resolve()

-1.0
-1.0
0.0
1.0
-1.0
0.0
0.0
0.0
1.0
-1.0
1.0
-1.0
0.0
1.0
0.0
-1.0
0.0
-1.0
0.0
1.0
True
